In [4]:
import matplotlib.pyplot as plt
import Portfolio
from dateutil import rrule 
import datetime
from datetime import timedelta as td
import time
import math
import numpy as np
import pandas as pd

In [6]:
tickers = ['YHOO','STK','GOOGL','AMZN','EBAY']
weights = [0.2,0.1,0.4,0.15,0.15]
#instantiate positions
positions = []
start_date = datetime.datetime(2010, 1, 1)
for i in range(len(tickers)):
    positions.append(Portfolio.Position(tickers[i],"long",30,start_date,weights[i],"NASDAQ","stock"))
#instantiate portfolio
portfolio = Portfolio.Portfolio(positions)
mu = portfolio.net_expectation
sigma = math.sqrt(portfolio.net_variance)

In [7]:
mu

0.055875472973644209

In [8]:
sigma

1.4162618350646003

In [121]:
n_trials = 600
horizon = 12
days, perf = monte_carlo_simulation(mu,sigma,n_trials,horizon)
def MonteCarlo_statistics(perf):
    #equity = (1+0.01r1)(1+0.01r2)...(1+0.01rn)
    final = 1+(perf/100)
    final = np.prod(final,axis=1)
    
    final = pd.DataFrame(final)
    info = final.describe()
    quartile1 = info[0]['25%']
    quartile2 = info[0]['50%']
    quartile3 = info[0]['75%']
    index_quartile1 = final[(quartile1-0.001*quartile1<final) & (quartile1+0.001*quartile1>final)].first_valid_index()
    index_quartile2 = final[(quartile2-0.001*quartile2<final) & (quartile2+0.001*quartile2>final)].first_valid_index()
    index_quartile3 = final[(quartile3-0.001*quartile3<final) & (quartile3+0.001*quartile3>final)].first_valid_index()
    
    perf_series_quartile1 = perf[index_quartile1]
    perf_series_quartile2 = perf[index_quartile2]
    perf_series_quartile3 = perf[index_quartile3]
    info.columns = ['Stats']
    return (info,perf_series_quartile1,perf_series_quartile2,perf_series_quartile3)

,Stats
count,600.000000
mean,1.141609
std,0.255400
min,0.586072
25%,0.965759
50%,1.108551
75%,1.293347
max,2.137497


In [146]:
#simply the cdf of the distribution of final performance obtained from monte carlo simulation
def probability_of_perf(final_perf,goal,upside=True):
    ordered_perfs = final_perf[0].sort(inplace=False)
    if upside is True:
        p = ordered_perfs[ordered_perfs >= goal].count()/(ordered_perfs.count()+1)
    else:
        p = ordered_perfs[ordered_perfs <= goal].count()/(ordered_perfs.count()+1)
    return p

In [1]:
#horizon in months
#about 45 seconds for 1 million trials and 4 years
def monte_carlo_simulation(mu,sigma,n_trials,horizon):
    start_date = datetime.date.today()
    end_date = start_date + datetime.timedelta(horizon*365/12)
    trading_days = list(NYSE_tradingdays(start_date,end_date))
    n_days = len(trading_days)
    returns = np.random.normal(mu,sigma,size=(n_trials,n_days))
    return trading_days,returns

In [2]:
def NYSE_tradingdays(a,b):
    rs = rrule.rruleset()
    rs.rrule(rrule.rrule(rrule.DAILY, dtstart=a, until=b))
    # Exclude weekends and holidays
    rs.exrule(rrule.rrule(rrule.WEEKLY, dtstart=a, byweekday=(rrule.SA,rrule.SU)))
    rs.exrule(NYSE_holidays(a,b))
    return rs 

In [5]:
def NYSE_holidays(a=datetime.date.today(), b=datetime.date.today()+datetime.timedelta(days=365)):
    rs = rrule.rruleset()
    # Include all potential holiday observances
    rs.rrule(rrule.rrule(rrule.YEARLY, dtstart=a, until=b, bymonth=12, bymonthday=31, byweekday=rrule.FR)) # New Years Day  
    rs.rrule(rrule.rrule(rrule.YEARLY, dtstart=a, until=b, bymonth= 1, bymonthday= 1))                     # New Years Day  
    rs.rrule(rrule.rrule(rrule.YEARLY, dtstart=a, until=b, bymonth= 1, bymonthday= 2, byweekday=rrule.MO)) # New Years Day    
    rs.rrule(rrule.rrule(rrule.YEARLY, dtstart=a, until=b, bymonth= 1, byweekday= rrule.MO(3)))            # Martin Luther King Day   
    rs.rrule(rrule.rrule(rrule.YEARLY, dtstart=a, until=b, bymonth= 2, byweekday= rrule.MO(3)))            # Washington's Birthday
    rs.rrule(rrule.rrule(rrule.YEARLY, dtstart=a, until=b, byeaster= -2))                                  # Good Friday
    rs.rrule(rrule.rrule(rrule.YEARLY, dtstart=a, until=b, bymonth= 5, byweekday= rrule.MO(-1)))           # Memorial Day
    rs.rrule(rrule.rrule(rrule.YEARLY, dtstart=a, until=b, bymonth= 7, bymonthday= 3, byweekday=rrule.FR)) # Independence Day
    rs.rrule(rrule.rrule(rrule.YEARLY, dtstart=a, until=b, bymonth= 7, bymonthday= 4))                     # Independence Day
    rs.rrule(rrule.rrule(rrule.YEARLY, dtstart=a, until=b, bymonth= 7, bymonthday= 5, byweekday=rrule.MO)) # Independence Day
    rs.rrule(rrule.rrule(rrule.YEARLY, dtstart=a, until=b, bymonth= 9, byweekday= rrule.MO(1)))            # Labor Day
    rs.rrule(rrule.rrule(rrule.YEARLY, dtstart=a, until=b, bymonth=11, byweekday= rrule.TH(4)))            # Thanksgiving Day
    rs.rrule(rrule.rrule(rrule.YEARLY, dtstart=a, until=b, bymonth=12, bymonthday=24, byweekday=rrule.FR)) # Christmas  
    rs.rrule(rrule.rrule(rrule.YEARLY, dtstart=a, until=b, bymonth=12, bymonthday=25))                     # Christmas  
    rs.rrule(rrule.rrule(rrule.YEARLY, dtstart=a, until=b, bymonth=12, bymonthday=26, byweekday=rrule.MO)) # Christmas 
    # Exclude potential holidays that fall on weekends
    rs.exrule(rrule.rrule(rrule.WEEKLY, dtstart=a, until=b, byweekday=(rrule.SA,rrule.SU)))
    return rs